In [18]:
import numpy as np # linear algebra
import pandas as pd

In [19]:
data = pd.read_csv('/content/drive/MyDrive/Data mining data set.csv')

In [20]:
data = data.drop('ProperCase', axis=1)
data = data.drop('IncGeo_WardName', axis=1)
data = data.drop('Easting_m', axis=1)
data = data.drop('Northing_m', axis=1)
data = data.drop('FRS', axis=1)
data = data.drop('AddressQualifier', axis=1)
data = data.drop('Postcode_full', axis=1)
data = data.drop('Postcode_district', axis=1)

In [21]:
data = data.drop('StopCodeDescription', axis=1)
data = data.drop('SpecialServiceType', axis=1)


In [22]:
data = data.dropna(axis=0, subset=['FirstPumpArriving_AttendanceTime'])
data = data.dropna(axis=0, subset=['FirstPumpArriving_DeployedFromStation'])
data = data.dropna(axis=0, subset=['PumpHoursRoundUp'])
data = data.dropna(axis=0, subset=['Notional Cost (£)'])

In [23]:
import pyproj

# Define the BNG CRS
bng = pyproj.Proj(proj='tmerc', ellps='airy', datum='OSGB36',
                  lat_0=49, lon_0=-2, x_0=400000, y_0=-100000)

# Define the WGS 84 CRS (used for latitude and longitude)
wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')

# Define your Easting and Northing coordinates
easting = data['Easting_rounded']
northing = data['Northing_rounded']

# Convert the coordinates
longitude, latitude = pyproj.transform(bng, wgs84, easting, northing)

print("Longitude:", longitude)
print("Latitude:", latitude)

data['Longitude'] = longitude
data['Latitude'] = latitude

<ipython-input-23-e4edb5d7b729>:15: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude, latitude = pyproj.transform(bng, wgs84, easting, northing)


Longitude: [-0.38135501 -0.22024751 -0.17837216 ... -0.33739497 -0.17507455
 -0.17100931]
Latitude: [51.55933488 51.51205787 51.65883095 ... 51.53894521 51.48889147
 51.48253589]


In [24]:
lambethdata = data[data['IncGeo_BoroughName'] == 'LAMBETH']

In [38]:
# assume that df is the DataFrame you want to save
lambethdata.to_csv('lambeth.csv', index=False)

In [39]:
lambethdata.shape

(18592, 27)

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lambethdata = lambethdata.drop('IncGeo_BoroughName', axis=1)
# Drop irrelevant columns and the target variable 'NumPumpsAttending'
# Keep columns that might be useful for prediction
X = lambethdata.drop(['IncidentNumber', 'DateOfCall', 'TimeOfCall', 'NumPumpsAttending'], axis=1)
y = lambethdata['NumPumpsAttending']

# Handle missing values (if any)
X = X.fillna(0)

# Encode categorical variables using one-hot encoding
categorical_cols = ['IncidentGroup', 'PropertyCategory', 'PropertyType', 'IncGeo_WardNameNew', 'IncGeo_BoroughCode', 'IncGeo_WardCode', 'IncidentStationGround', 'FirstPumpArriving_DeployedFromStation', 'SecondPumpArriving_DeployedFromStation']
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Scale numerical features using StandardScaler
numerical_cols = ['CalYear', 'HourOfCall', 'FirstPumpArriving_AttendanceTime', 'SecondPumpArriving_AttendanceTime', 'NumStationsWithPumpsAttending', 'PumpCount', 'PumpHoursRoundUp', 'Notional Cost (£)']
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# Train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)



Mean Squared Error: 0.039741227723573726
R-squared: 0.952077918786699


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Drop irrelevant columns and the target variable 'NumPumpsAttending'
# Keep columns that might be useful for prediction
X = lambethdata.drop(['IncidentNumber', 'DateOfCall', 'IncidentGroup', 'TimeOfCall'], axis=1)
y = lambethdata['IncidentGroup']

# Handle missing values (if any)
X = X.fillna(0)
y_encoded = pd.get_dummies(y)

# Encode categorical variables using one-hot encoding
categorical_cols = ['PropertyCategory', 'PropertyType', 'IncGeo_WardNameNew', 'IncGeo_BoroughCode', 'IncGeo_WardCode', 'IncidentStationGround', 'FirstPumpArriving_DeployedFromStation', 'SecondPumpArriving_DeployedFromStation']
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Scale numerical features using StandardScaler
numerical_cols = ['CalYear', 'HourOfCall', 'FirstPumpArriving_AttendanceTime', 'SecondPumpArriving_AttendanceTime', 'NumStationsWithPumpsAttending', 'PumpCount', 'PumpHoursRoundUp', 'Notional Cost (£)']
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)


Accuracy: 0.8047862328582952
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.87      0.87      1887
           1       0.77      0.52      0.62       581
           2       0.84      0.84      0.84      1251

   micro avg       0.85      0.80      0.83      3719
   macro avg       0.83      0.74      0.78      3719
weighted avg       0.84      0.80      0.82      3719
 samples avg       0.80      0.80      0.80      3719



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
data.head()

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,PropertyCategory,PropertyType,UPRN,USRN,...,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpHoursRoundUp,Notional Cost (£)
0,000221-01012017,1/1/2017,2017,9:55:49,9,Special Service,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,0,21201921,...,Northolt,372.0,Northolt,415.0,Harrow,2.0,2.0,2.0,1.0,326.0
1,000233-01012017,1/1/2017,2017,10:39:29,10,Fire,Outdoor Structure,Barbecue,217127955,21701312,...,North Kensington,379.0,North Kensington,469.0,Kensington,2.0,2.0,2.0,1.0,326.0
2,000243-01012017,1/1/2017,2017,11:04:10,11,Special Service,Outdoor,Woodland/forest - broadleaf/hardwood,200149180,20017170,...,Barnet,397.0,Barnet,NaN,NaN,1.0,1.0,1.0,1.0,326.0
3,000304-01012017,1/1/2017,2017,14:32:26,14,False Alarm,Dwelling,House - single occupancy,0,21500857,...,Heston,315.0,Heston,318.0,Heston,2.0,3.0,3.0,1.0,326.0
4,000257-01012017,1/1/2017,2017,12:14:35,12,False Alarm,Dwelling,Converted Flat/Maisonettes - 3 or more storeys,0,20900933,...,Stoke Newington,378.0,Homerton,392.0,Homerton,2.0,3.0,3.0,2.0,652.0
